In [1]:
import pandas as pd
import numpy as np
from haversine import haversine
import datetime
import os
import json

zip_centers=json.load(open("/home/jian/Docs/Geo_mapping/updated_zip_centers_JL_2019-05-23.json","r"))

In [2]:
target_df=pd.read_excel("./BL SOTF 7.26 Wave Zips - updated 6.10.xlsx",dtype=str)
target_store_list=target_df['Store ID'].unique().tolist()

In [3]:
store_label=pd.read_csv("/home/jian/Projects/Big_Lots/New_TA/zips_in_new_ta/sales_by_zip (Store level).csv",dtype=str)
store_label=store_label[store_label['location_id'].isin(target_store_list)]

In [6]:
store_list=pd.read_table("/home/jian/BigLots/static_files/Store_list/MediaStormStores20190601-134302-813.txt",
                         dtype=str,sep="|")
store_list=store_list[['location_id','latitude_meas','longitude_meas']]
store_list['latitude_meas']=store_list['latitude_meas'].astype(float)
store_list['longitude_meas']=store_list['longitude_meas'].astype(float)
store_list=store_list[store_list['revenue_flag']!="T"]

In [7]:
store_label=pd.merge(store_label,store_list,on="location_id",how="left")

In [9]:
store_label=store_label.reset_index()
del store_label['index']

In [10]:
store_label.head(2)

,net_transaction_amt,transactions,zip,revenue_flag,transaction_flag,location_id,latitude_meas,longitude_meas
0,305640.33,431,37814,P,P,1052,36.224703,-83.265513
1,177894.6,382,37813,P,P,1052,36.224703,-83.265513


In [14]:
store_label['dist']=store_label.apply(lambda x: haversine([x['latitude_meas'],x['longitude_meas']],zip_centers[x['zip']],miles=True)
                                      

SyntaxError: unexpected EOF while parsing (<ipython-input-14-bc0787df0b5a>, line 2)

In [17]:
import zipcodes
zipcodes.matching('10044')[0]

{'active': True,
 'city': 'NEW YORK',
 'country': 'US',
 'lat': 40.71,
 'long': -73.99,
 'state': 'NY',
 'world_region': 'NA',
 'zip_code': '10044',
 'zip_code_type': 'STANDARD'}

In [18]:
import zipcodes

store_label['dist']=np.nan
for index,row in store_label.iterrows():
    store_center=[row['latitude_meas'],row['longitude_meas']]
    zip_cd=row['zip']
    if zip_cd in zip_centers.keys():
        zip_center_row=zip_centers[zip_cd]
        dist=haversine(store_center,zip_center_row,miles=True)
        store_label.loc[index,'dist']=dist
    else:
        try:
            zip_center_row=[zipcodes.matching(zip_cd)[0]['lat'],zipcodes.matching(zip_cd)[0]['long']]
            dist=haversine(store_center,zip_center_row,miles=True)
            store_label.loc[index,'dist']=dist
        except:
            dist=np.nan

In [21]:
output_old_lables=store_label[(store_label['revenue_flag']=="P") | (store_label['dist']<=20)]

In [34]:
.shape

(1029, 9)

In [27]:
df_cc=pd.read_excel("./output_Charles.xlsx",dtype=str)

In [25]:
os.getcwd()

'/home/jian/Projects/Big_Lots/Analysis/2019_Q2/Planner_Requests/zips_Faith_SOTF_20190610'

In [29]:
output_JL=output_old_lables[['location_id','zip','revenue_flag']].drop_duplicates()
output_JL['JL']="JL"

df_cc['CC']="CC"

In [32]:
QC_DF=pd.merge(output_JL,df_cc,on=["location_id",'zip','revenue_flag'],how="outer")

In [33]:
QC_DF

,location_id,zip,revenue_flag,JL,G.O.,store_zip,latitude_meas,longitude_meas,Distance (mile),CC
0,1052,37814,P,JL,2019-06-20,37814,36.22470300000001,-83.265513,2.524753982075854,CC
1,1052,37813,P,JL,2019-06-20,37814,36.22470300000001,-83.265513,2.379581158426659,CC
2,1052,37708,S,JL,2019-06-20,37814,36.22470300000001,-83.265513,6.880019171107341,CC
3,1052,37860,P,JL,2019-06-20,37814,36.22470300000001,-83.265513,4.296764729108864,CC
4,1052,37891,S,JL,2019-06-20,37814,36.22470300000001,-83.265513,8.439823878389916,CC
5,1052,37821,P,JL,2019-06-20,37814,36.22470300000001,-83.265513,18.33611159147197,CC
6,1052,37877,S,JL,2019-06-20,37814,36.22470300000001,-83.265513,9.486212615434468,CC
7,1052,37890,S,JL,2019-06-20,37814,36.22470300000001,-83.265513,9.392327848782859,CC
8,1052,37811,S,JL,2019-06-20,37814,36.22470300000001,-83.265513,9.014171895763406,CC
9,1052,37711,S,JL,2019-06-20,37814,36.22470300000001,-83.265513,13.38985506710608,CC
